Seems to me that instead of trying to match LCBO products to reddit reviews, it might make more sense to match reddit reviews to LCBO products, since those are the set we care more about.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [4]:
import praw
import pandas as pd
import re

import requests
import time
import sys
import pdb

# Load Data

In [5]:
reviews = pd.read_parquet('data/db_reviews.parquet')

/opt/tljh/user/lib/python3.6/site-packages/pyarrow/pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/opt/tljh/user/lib/python3.6/site-packages/pyarrow/pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
/opt/tljh/user/lib/python3.6/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [6]:
whiskyids = pd.read_parquet('data/whisky_ids.parquet')

In [7]:
lcbo = pd.read_parquet('data/lcbo_whisky.parquet').drop_duplicates()

Let's load the table of one's we've already matched up.

In [8]:
onematch = pd.read_parquet('data/onematch.parquet')

# Join Datasets

Assign our unique IDs to the reviews table:

In [9]:
reviews = (reviews.reset_index()
                  .set_index('whisky')
                  .join(whiskyids.set_index('RedditWhiskyName'))
                  .reset_index()
                  .rename({'index':'whisky'}, axis='columns')
          )

Now join LCBO to matches:

In [15]:
onematch.set_index('ProductID')

,Name,RedditWhiskyID,RedditWhiskyName
ProductID,,,
496729,1792 SINGLE BARREL KENTUCKY STRAIGHT BOURBON W...,15,1792 Single Barrel
208918,1792 SMALL BATCH KENTUCKY STRAIGHT BOURBON,19,1792 Small Batch
634519,601 BOURBON,25,601 Bourbon
255281,ABERFELDY 12 YEAR OLD,34,Aberfeldy 12
255281,ABERFELDY 12 YEAR OLD,35,Aberfeldy 12 Year Old
400085,ABERFELDY 21 YEAR OLD HIGHLAND SINGLE MALT SCO...,46,Aberfeldy 21
482885,ABERLOUR 10 YEAR OLD SINGLE MALT SCOTCH WHISKY,48,Aberlour 10
573352,ABERLOUR A'BUNADH SCOTCH WHISKY,83,Aberlour A'bunadh
216226,ALBERTA PREMIUM WHISKY,168,Alberta Premium 30


In [11]:
lcbo.set_index('itemnumber').join(onematch.set_index('ProductID'))

,language,itemname,price,productsize,isdiscontinued,islimited,am,ambonusmiles,amexpiration,kosher,...,winestyle,winevarietal,stylebody,stylebody_fr,styleflavour,styleflavour_fr,quantity,Name,RedditWhiskyID,RedditWhiskyName
42,en,CANADIAN CLUB WHISKY,27.20,750,True,True,False,0,NaT,False,...,Subtle & Sweet,Canada,Subtle,Léger,Sweet,doux,1,NaN,NaN,NaN
281,en,WALKER'S SPECIAL OLD WHISKY,27.45,750,True,True,False,0,NaT,False,...,Subtle & Spicy,Canada,Subtle,Léger,Spicy,épicé,1,NaN,NaN,NaN
307,en,SEAGRAMS V.O. WHISKY,27.20,750,True,True,False,0,NaT,False,...,Subtle & Spicy,Canada,Subtle,Léger,Spicy,épicé,1,NaN,NaN,NaN
463,en,CANADIAN CLUB WHISKY,15.75,375,True,True,False,0,NaT,False,...,Subtle & Sweet,Canada,Subtle,Léger,Sweet,doux,1,NaN,NaN,NaN
620,en,SILK TASSEL CANADIAN WHISKY (PET),27.25,750,True,True,False,0,NaT,False,...,Medium & Sweet,Canada,Medium,Mi-corsé,Sweet,doux,1,NaN,NaN,NaN
687,en,WALKER'S SPECIAL OLD WHISKY (PET),15.45,375,True,True,False,0,NaT,False,...,Subtle & Spicy,Canada,Subtle,Léger,Spicy,épicé,1,NaN,NaN,NaN
893,en,J.P. WISER'S DELUXE WHISKY,27.20,750,True,True,False,0,NaT,False,...,Subtle & Sweet,Canada,Subtle,Léger,Sweet,doux,1,NaN,NaN,NaN
984,en,ALBERTA PREMIUM WHISKY,27.20,750,True,True,True,10,2019-07-20,False,...,Medium & Spicy,Canada,Medium,Mi-corsé,Spicy,épicé,1,NaN,NaN,NaN
992,en,ROYAL RESERVE WHISKY (PET),40.90,1140,True,True,False,0,NaT,False,...,Subtle & Sweet,Canada,Subtle,Léger,Sweet,doux,1,NaN,NaN,NaN
1099,en,JOHNNIE WALKER RED LABEL SCOTCH WHISKY,32.20,750,True,True,False,0,NaT,False,...,Subtle & Smoky,Scotland Blend,Subtle,Léger,Smoky,fumé,1,NaN,NaN,NaN


In [ ]:
lcbomatch = lcbo.

In [175]:
reviews = reviews.set_index('RedditWhiskyID').join(onematch.set_index('RedditWhiskyID')).reset_index().drop('RedditWhiskyName',axis='columns').rename({'Name':'LCBOName'},axis='columns')

# Check Coverage

In [176]:
reviews.shape

(31282, 14)

How many have we matched up?

In [177]:
reviews[reviews.RedditWhiskyID>0].shape

(30588, 14)

In [178]:
len(reviews.whisky.unique())

10169

In [192]:
reviews.head()

,RedditWhiskyID,whisky,reviewID,timestamp,username,reviewLink,rating,style,pricePaid,date,sub_id,review,LCBOName,ProductID
0,0.0,100 Pipers,0,2012-12-14 10:03:18.000,merlinblack,http://www.reddit.com/r/Scotch/comments/14uder...,68,Blend,nan,2012-12-14,14uder,My wife and I are on a trip to Thailand to mee...,NaN,NaN
1,2.0,1792 Bottled In Bond,3,2018-11-04 00:49:14.921,scottmotorrad,https://www.reddit.com/r/bourbon/comments/9ryi...,80,Bourbon,34.99,2018-10-27,9ryi8f,The 1792 review series continues! The Bottled ...,NaN,NaN
2,3.0,1792 Bottled In Bond Oak Liquor Cabinet Pick,4,2018-12-19 00:38:02.415,scottmotorrad,https://www.reddit.com/r/bourbon/comments/a7jw...,90,Bourbon,nan,2018-12-19,a7jwdd,This week Oak Liquor Cabinet got in the long a...,NaN,NaN
3,4.0,1792 Full Proof,5,2016-10-31 16:14:04.778,dmsn7d,https://www.reddit.com/r/bourbon/comments/5aez...,85,Bourbon,nan,2016-10-31,5aezxj,**1792 Full Proof**\n\n***\n\n**Price:** appro...,NaN,NaN
4,4.0,1792 Full Proof,6,2017-06-29 15:44:31.555,FuzzyWildcat,https://www.reddit.com/r/bourbon/comments/6epe...,69,Bourbon,34,2017-06-01,6epezn,"Also as a quick note, this stuff is however pr...",NaN,NaN


In [200]:
reviews.dropna(subset=['ProductID']).shape[0]

6889

In [219]:
def showAmountMatched(reviews):
    print('Matched ' + \
      str(reviews.dropna(subset=['ProductID']).shape[0]) + \
      ' of ' + \
      str(len(reviews)) + \
      ' reviews : ' + \
     str(round(100*reviews.dropna(subset=['ProductID']).shape[0]/len(reviews),3)) + \
      ' %'
     )
    print('Matched ' + \
      str(len(reviews.dropna(subset=['ProductID']).whisky.unique())) + \
      ' of ' + \
       str(reviews.dropna(subset=['ProductID']).shape[0]) + \
      ' whiskies : ' + \
     str(round(100*len(reviews.dropna(subset=['ProductID']).whisky.unique())/len(reviews.whisky.unique()),3)) + \
      ' %'
     )

In [220]:
showAmountMatched(reviews)

Matched 6889 of 30979 reviews : 22.238 %
Matched 526 of 6889 whiskies : 5.52 %


We have very low coverage. Let's see if we can increase this.

# Additional Data Matchup

Let's see how many unique we have, and take a look.

In [181]:
len(reviews.whisky.unique())

10169

In [182]:
[print(i) for i in reviews.whisky.unique()]

100 Pipers
1792 Bottled In Bond
1792 Bottled In Bond Oak Liquor Cabinet Pick
1792 Full Proof
1792 Full Proof Angel's Beverage
1792 Full Proof Binny's
1792 Full Proof Loch & K(e)y Microbatch
1792 Full Proof Lueken's Store Pick
1792 Full Proof Oak Liquor Cabinet Pick
1792 Full Proof Poison Girl
1792 Full Proof Red Dog Wine & Spirits
1792 High Rye
1792 Port Finish
1792 Ridgemont Reserve
1792 Single Barrel
1792 Single Barrel (Crown Liquors, 7/12/16 Bottle)
1792 Single Barrel Oak Liquor Cabinet Pick
1792 Single Barrel Parkway Wine And Spirits
1792 Small Batch
1792 Sweet Wheat
1835 Bourbon Whiskey
291 Single Barrel Colorado Rye Whiskey
4 Roses PS OESK
601 Bourbon
66 Gilead Crimson Rye
66 Gilead Wild Oak
A Drop of the Irish Black Adder 2016 Sherry Cask Finish Single Cask
A.D. Laws Four Grain Straight Bourbon
A.D. Laws Four Grain Straight Bourbon BiB
A.D. Laws Secale Straight Rye
A.H. Hirsch 16
A.H. Hirsch 16 gold foil
Aberfeldy 12
Aberfeldy 12 Year Old
Aberfeldy 16 "Ramble" Single Cask
Aberfe

Ben Nevis 16 Signatory
Ben Nevis 17 1996 Maltman fino sherry
Ben Nevis 17 1997 Carn Mor
Ben Nevis 17 1997 Sherry Butt (Carn Mor)
Ben Nevis 17 A.D. Rattray
Ben Nevis 17 Alambic Classique
Ben Nevis 17 Cadenhead's
Ben Nevis 17 Exclusive Malts
Ben Nevis 17 The Maltman
Ben Nevis 18 (Cooper’s Choice 1996)
Ben Nevis 18 1995 Private Bottling
Ben Nevis 18 1996 Coopers Choice
Ben Nevis 18 1996 The Maltman
Ben Nevis 18 1997 John Milroy
Ben Nevis 18 1997 John Milroy K&L Exclusive Single Barrel Cask Strength
Ben Nevis 18 First Editions
Ben Nevis 19 1997 Jack Wieber
Ben Nevis 19 Sherry
Ben Nevis 19 Signatory CS #2305
Ben Nevis 19 TBWC
Ben Nevis 19 The Whisky Agency Good Vibes 1996
Ben Nevis 1991 Signatory
Ben Nevis 1992
Ben Nevis 1992 Cadenhead's
Ben Nevis 20 (Sansibar 1996)
Ben Nevis 20 1996
Ben Nevis 20 1996 Archives
Ben Nevis 20 1996 Old Particular
Ben Nevis 20 1996 Sansibar
Ben Nevis 20 1996 Single Cask Nation
Ben Nevis 20, refill oloroso puncheon Single Cask Nation
Ben Nevis 2006 10yo (Le Gus't

Blue Hanger 7th Release
Blue Hanger 9th Release
Bluegrass Distillers
Bomb - Pedro Ximenez Cask Finish Whisky Lounge
Bombergers Declaration 2018
Bond + Lillard
Booker's 25th Anniversary
Booker's 30th
Booker's 30th Anniversary
Booker's Bourbon
Booker's Bourbon 2013-6 Roundtable Batch
Booker's Bourbon 2014-06
Booker's Bourbon 2016-04
Booker's Bourbon 2017-4 (Sip Awhile)
Booker's Bourbon Batch 2016-01E
Booker's Bourbon Big Man Small Batch
Booker's Bourbon Bluegill Creek
Booker's Bourbon C97-A-31
Booker's Bourbon Center Cut
Booker's Bourbon Maw Maw's Batch
Booker's Bourbon Noe Secret
Booker's Bourbon Off Your Rocker
Booker's Bourbon Oven Buster
Booker's c2014-05
Booker's Kathleen's Batch
Booker's Little Book Chapter 2: Noe Simple Task
Booker's Rye
Booker’s Rye 13 1/12 12/365
Booker’s Rye Big Time Batch
Boone County 12 Single Barrel Liquor Barn
Boone County 1833
Boone County Single Barrel 12 Year - Liquor Barn Pick
Bootlegger 21 New York Bourbon
Border Straight Bourbon
Bourbon deLuxe 1993
Bo

Bruichladdich Port Charlotte 9 Sherry
Bruichladdich Port Charlotte 9 Single Cask Private Cask 933
Bruichladdich Port Charlotte An Turas Mor
Bruichladdich Port Charlotte Archive 2001
Bruichladdich Port Charlotte Cask Exploration 12 -Tunn
Bruichladdich Port Charlotte Elements of Islay PL4
Bruichladdich Port Charlotte Islay Barley
Bruichladdich Port Charlotte Islay Barley 2008
Bruichladdich Port Charlotte MC:01
Bruichladdich Port Charlotte MC:01 2009
Bruichladdich Port Charlotte Micro Provenance Cask #0005 - Virgin Oak
Bruichladdich Port Charlotte Micro Provenance Cask #0013 - Bordeaux
Bruichladdich Port Charlotte Micro Provenance Cask #1999 - Bourbon
Bruichladdich Port Charlotte MRC:01
Bruichladdich Port Charlotte PC10
Bruichladdich Port Charlotte PC10 Tro Na Linntean
Bruichladdich Port Charlotte PC11 Eorna Na H-Alba
Bruichladdich Port Charlotte PC12
Bruichladdich Port Charlotte PC12 Oileanach Furachail
Bruichladdich Port Charlotte PC6
Bruichladdich Port Charlotte PC6 Cuairt-Beatha
Bruic

Clynelish 21 Dun Bheagan
Clynelish 21 Signatory CS for The Whisky Exchange
Clynelish 21 Signatory Vintage TWE Cask 8676
Clynelish 21 SMWS 26.119
Clynelish 21 SMWS 26.119 "Dark and dangerously good"
Clynelish 21 The Ultimate
Clynelish 21 TWE Show Bottling 2017
Clynelish 22 1995 Signatory Vintage CS Collection Refill Sherry Butt (#11230)
Clynelish 23 SMWS 26.99 "The Fisherman's Friend"
Clynelish 24 1988 The First Editions
Clynelish 24 1989 Adelphi Cask 3846
Clynelish 24 1990 Adelphi Cask 3232
Clynelish 24 The First Editions
Clynelish 25 1982 AD Rattray
Clynelish 25 SMWS 26.67 "Candlewax and red berries"
Clynelish 26 1988(Càrn Mòr Celebration of the Cask
Clynelish 28 1972 Adelphi
Clynelish 29 1984 SMWS 26.105 "Bumblebees by the sea"
Clynelish 29 1984 SMWS 26.106 "Say it with flowers"
Clynelish 29 SMWS 26.102 "Pomanders in a Lady's Parlour"
Clynelish 29 SMWS 26.105 "Bumblebees by the Sea"
Clynelish 29 SMWS 26.106
Clynelish 44 Gordon MacPhail Rare Old 1972
Clynelish 6 2008 The Ultimate
Clyn

...

Old Weller Antique 107, Lincoln Road
Old Weller Antique 107, McScrooges
Old Weller Antique 107, Red Dog
Old Weller Antique 107, TPS
Old Weller Antique 107, TPS Select
Old Weller Antique 2006 Paper Label
Old Weller Antique Chuck's Liquor
Old Weller Antique Gonomo
Old Weller Antique Lock & Keye Non-Chill Filtered
Old Weller Antique Private Select - "The Party Source"
Old Weller Antique Red Dog
Old Weller Antique SiB - Kappy's
Old Weller Antique Single Barrel
Old Weller Antique Store Pick
Old Weller Centennial 10 Louisville Stitzel-Weller
Omar Nantou Sherry SC Cask #21130028
One Blended Whisky Sherry Ed.
One Eight Distilling Untitled Whiskey No. 1
One Eight Untitled #2
Oola Waitsburg Bourbon Whiskey
Oppidan Malted Rye
Orbis Aged World Whiskey
Orbital 8
Ordha 21
Oregon Spirits Bourbon
Original Jesse James
Origine
Origine 10
Orkney 17 2000 The Whisky Agency Toronto Whisky Society Pick)
Orkney 17 2018 The Whisky Agency & Heads & Tails
Orphan Barrel Barterhouse
Orphan Barrel Barterhouse 20
Or

Talisker Distillers Edition
Talisker Distillers Edition 2007-2017
Talisker Neist Point
Talisker Port Ruighe
Talisker Skye
Talisker Storm
Talisker Young & Feisty - Provenance DL
Talisman
Tamdhu 10
Tamdhu 10 2005 Signatory Vintage First Fill Sherry
Tamdhu 10 First Fill Sherry
Tamdhu 10 Signatory Cask Strength
Tamdhu 10 Signatory CS 2005
Tamdhu 10 Signatory First Fill Sherry
Tamdhu 10 Signatory Vintage Binny's Selection
Tamdhu 10 Single Cask Nation
Tamdhu 17 1995 Adelphi
Tamdhu 20 1998 Old Malt Cask K&L Wines
Tamdhu 2008 Carn Mor
Tamdhu 25 1988 Old Malt Cask
Tamdhu 30 MacPhail's Collection
Tamdhu 34 Duncan Taylor
Tamdhu 6 The Ultimate
Tamdhu 8 2006 Hepburns Choice
Tamdhu 8 MacPhail's Collection
Tamdhu 8 Signatory Cask Strength
Tamdhu 8 The Ultimate
Tamdhu 9 2006 The Ultimate
Tamdhu Batch Strength
Tamdhu Batch Strength 001
Tamdhu Batch Strength 002
Tamdhu Batch Strength 003
Tamnavulin 17 Connoisseur's Choice
Tamnavulin 1993 Old Malt Cask
Tamnavulin 21 Single Malts of Scotland
Tamnavulin 22

Wemyss The Hive
Wemyss The Peat Chimney
Wemyss Velvet Fig
West Cork 10 Single Malt
West Cork Cask Strength
West Cork Original
Westchester Pot Distilled Wheat Whiskey
Westland 2 Single Cask Nation Cask #736
Westland 5 SMWS 133.1
Westland 5 SMWS 133.1 "Speakeasy sneaky peaky"
Westland 54 Month New Oak Single Malt
Westland American Oak
Westland American Single Malt
Westland American Single Malt, Cask No. 115
Westland American Whiskey
Westland Cask 405
Westland Cask 606
Westland Garryana
Westland Garryana 2 - 2017
Westland Garryana 2016
Westland Garryana 2016 Edition 1
Westland Garryana 3 - 2018
Westland Handfilled Single Cask #448
Westland Peat Week 2016
Westland Peated
Westland Sherry Wood
Westland Single Cask
Westland Single Cask #115 Garryana Oak
Westland Single Cask #118
Westland Single Cask #196
Westland Single Cask #266
Westland Single Cask #2783
Westland Single Cask #300
Westland Single Cask #300 Fino Sherry
Westland Single Cask #350 First fill bourbon
Westland Single Cask #582 Fir

Glen Grant 2007, SMWS 9.161 “Cream Tea at the Patisserie” 
Glen Grant 21 Adelphi 
Glen Grant 36 Old and Rare 1975 Douglas Laing 
Glen Moray 13 SMWS 35.165 "Variety - the spice of life" 
Glen Moray Elgin Classic 
Glen Ord 13 Cadenhead's 
Glen Scotia 12 
Glen Scotia 12 2003 Single Cask Distillery Edition 002 
Glen Scotia 22 
Glen Scotia 24 Hart Brothers Madeira 
Glen Scotia SMWS 93.63 
Glen Spey 12 
GlenDronach 8 Octarine 
GlenDronach 8 SMWS 96.9 Icy-Chilly 
GlenDronach Peated 
Glendronach 18 
Glenfarclas 10 
Glenfarclas 105 Cask Strength 20 
Glenfarclas 12 
Glenfarclas 14 
Glenfarclas 1993 Oloroso Sherry Cask 
Glenfarclas 35 1981 Port Cask 
Glenfarclas 40 
Glenfiddich 12 
Glenfiddich 14 Rich Oak 
Glenfiddich 19 Age of Discovery Bourbon Cask 
Glenfiddich 24 Blackadder 
Glenfiddich Experimental Series 21 Winter Storm 
Glenglassaugh Revival 
Glengoyne 10 
Glengoyne 14 1996 
Glengoyne 16 SMWS 123.9 
Glenkinchie 20 
Glenlivet 11 FF Sherry Signatory un-Chillfiltered Collection 
Glenlivet 19 S

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

# Clean whisky names

I think we can narrow this down, a lot of them are duplicates.

Trim whitespace

In [183]:
reviews['whisky'] = reviews['whisky'].str.strip()

In [184]:
len(reviews.whisky.unique())

9976

In [185]:
[print(i) for i in reviews.whisky.unique()]

100 Pipers
1792 Bottled In Bond
1792 Bottled In Bond Oak Liquor Cabinet Pick
1792 Full Proof
1792 Full Proof Angel's Beverage
1792 Full Proof Binny's
1792 Full Proof Loch & K(e)y Microbatch
1792 Full Proof Lueken's Store Pick
1792 Full Proof Oak Liquor Cabinet Pick
1792 Full Proof Poison Girl
1792 Full Proof Red Dog Wine & Spirits
1792 High Rye
1792 Port Finish
1792 Ridgemont Reserve
1792 Single Barrel
1792 Single Barrel (Crown Liquors, 7/12/16 Bottle)
1792 Single Barrel Oak Liquor Cabinet Pick
1792 Single Barrel Parkway Wine And Spirits
1792 Small Batch
1792 Sweet Wheat
1835 Bourbon Whiskey
291 Single Barrel Colorado Rye Whiskey
4 Roses PS OESK
601 Bourbon
66 Gilead Crimson Rye
66 Gilead Wild Oak
A Drop of the Irish Black Adder 2016 Sherry Cask Finish Single Cask
A.D. Laws Four Grain Straight Bourbon
A.D. Laws Four Grain Straight Bourbon BiB
A.D. Laws Secale Straight Rye
A.H. Hirsch 16
A.H. Hirsch 16 gold foil
Aberfeldy 12
Aberfeldy 12 Year Old
Aberfeldy 16 "Ramble" Single Cask
Aberfe

Balblair 2002
Balblair 2003
Balblair 2003 12
Balblair 2003 1st Release
Balblair 2004
Balblair 2004 1st Release
Balblair 2005
Balblair 2005 1st Edition
Balblair 2005 1st edition
Balblair 2005 1st Release
Balblair 21 1993 Gordon & MacPhail
Balblair 24 (G&M Cask Strength 1993)
Balblair 24 1993 Gordon & MacPhail Cask Strength
Balblair 26 1990 2nd Release
Balblair 27 1991 3rd Release
Balblair 9 SMWS 70.10
Balblair 90 2nd edition
Balcones Baby Blue
Balcones Blue Corn Bourbon
Balcones Brimstone
Balcones Fr.Oak
Balcones Rumble
Balcones Rumble Cask Reserve
Balcones Rye
Balcones Single Malt
Balcones Texas Rye
Balcones Texas Single Malt
Balcones The Fox
Balcones True Blue
Balcones True Blue 100
Balcones True Blue Cask Strength
Ballantine's 12
Ballantine's 12 Gold Seal
Ballantine's 12 Late 70's bottle
Ballantine's 12 Pure Malt
Ballantine's 17
Ballantine's 21
Ballantine's Finest
Ballantine's Glenburgie 15
Ballechin #7 Bordeaux Edradour
Ballechin 10
Ballechin 10 2007 Signatory
Ballechin 10 K&L Cask 

Benromach 2003
Benromach 2003 Origins Batch 4
Benromach 2008 Single Cask
Benromach 2008 Single Cask Ex-bourbon
Benromach 2008 Single Cask for Itlay
Benromach 23 Gordon & MacPhail 1980
Benromach 25 Cask Strength
Benromach 30
Benromach 35
Benromach 5
Benromach 7 1990 Ex-Sherry Cask
Benromach 8 2008 Van Wees Store Select
Benromach 9 2008 Single Cask Bottling for Whiskysite.nl
Benromach 9 2008 Whiskysite Store Select
Benromach Cask Strength
Benromach Chateau Cissac 2009
Benromach Hermitage Finish
Benromach Organic
Benromach Origins 2 Port Pipes
Benromach Origins Batch 1
Benromach Origins Batch 4
Benromach Peat Smoke
Benromach Peat Smoke 2002
Benromach Peat Smoke 2004
Benromach Peat Smoke 2005
Benromach Peat Smoke 2006
Benromach Peat Smoke 2008
Benromach Portwood
Benromach Sassicaia Finish
Benromach Traditional
Benromach Triple Distilled 2009
Berkshire Mountain Distillers - Sam Adams Cask Finish
Berkshire Mountain Distillers - Troegenator finished Bourbon
Berkshire Mountain Distillers Smoke

Bunnahabhain 39 1976 Cadenheads Small Batch
Bunnahabhain 39 Cadenhead 1976
Bunnahabhain 40 1963
Bunnahabhain 40 1968 Peerless Duncan Taylor
Bunnahabhain 43 1967 Malts of Scotland cask #3315
Bunnahabhain 6 2000 Berry Bros. & Rudd
Bunnahabhain 6 Moine
Bunnahabhain 7 2006 (cask 800089) Berry Bros & Rudd
Bunnahabhain 7 2007 Douglas of Drumlanrig
Bunnahabhain 8 2006 SMWS 10.90 "Muckle, massive intensity of smoke"
Bunnahabhain 8 2006 The Ultimate
Bunnahabhain 8 2007 Gordon & MacPhail
Bunnahabhain 8 2007 Hepburn's Choice Single Cask
Bunnahabhain 8 2009 Gordon & Macphail Cask Strength
Bunnahabhain 8 Battlehill
Bunnahabhain 8 Douglas Laing Provenance
Bunnahabhain 8 Gordon & MacPhail The MacPhail's Collection
Bunnahabhain 8 Gordon & MacPhail's Heavily Peated
Bunnahabhain 8 SMWS 10.104 "Wild and untamed"
Bunnahabhain 8 SMWS 10.90 Muckle, Massive Intensity of Smoke
Bunnahabhain 8 The MacPhail's Collection
Bunnahabhain 8 Valinch Red Wine Cask
Bunnahabhain 8 year Rioja Single Cask
Bunnahabhain 9 (Ba

Clynelish 16 1997 Signatory
Clynelish 16 Duncan Taylor Rare Auld
Clynelish 16 Oloroso Finish Darkness
Clynelish 16 Signatory CS #12797
Clynelish 17
Clynelish 17 1996 Exclusive Casks
Clynelish 17 1996 First Editions
Clynelish 17 1997 Adelphi Selection
Clynelish 17 1997 Signatory
Clynelish 17 1998 Signatory Vintage
Clynelish 17 A.D. Rattray
Clynelish 17 AD Rattray
Clynelish 17 Exclusive Casks
Clynelish 17 First Editions
Clynelish 17 Signatory
Clynelish 17 Signatory TWE Pick
Clynelish 18 (Whiskies of Scotland)
Clynelish 18 1995 Duncan Taylor Dimensions
Clynelish 18 1996 Signatory
Clynelish 18 1996 Signatory Cask Strength
Clynelish 18 1996 whiskybroker
Clynelish 18 1997 Old Particular
Clynelish 18 1997 Wilson & Morgan Marsala Finish
Clynelish 18 Hepburn's Choice
Clynelish 18 Hepburns Choice
Clynelish 18 Signatory
Clynelish 18 Signatory CS 1996
Clynelish 18 TWE
Clynelish 18 Whiskies of Scotland 1995
Clynelish 19 1996 Adelphi Selection Cask #11446
Clynelish 19 1996 Carn Mor
Clynelish 19 1996

...

Oamaruvian 16 Cask Strength Double Wood
Oban 14
Oban 18
Oban 18 Limited Edition
Oban Distiller's Edition
Oban Distiller's Edition 1995
Oban Distiller's Edition 1996
Oban Distiller's Edition 1997
Oban Distiller's Edition 2013
Oban Distiller's Edition 2015
Oban Distiller's Edition 2017
Oban Distillery Only Bottling 2016
Oban Game of Thrones The Night's Watch
Oban Little Bay
Octomore 10 2nd Edition
Octomore 7.2
Octomore 8.1
Octomore 9.3
Ohishi Rice Whisky Sherry Cask
Ohishi Sherry Cask
Ohishi Single Sherry Cask
OId Pulteney Navigatio
Oishii Wisukii Batch 2
Okanagan Spirits BRBN Corn Whisky
Okanagan Spirits Laird of Fintry
Okanagan Spirits Laird of Fintry 2015
Okanagan Spirits Laird of Fintry 2016
Okanagan Spirits Laird of Fintry Batch 5
Okanagan Spirits Laird of Fintry Single Malt Lot 4
Okanagan Spirits Rye Whisky
OKI 10 Year Winking Lizard Store Pick
OKI 12 Year Single Barrel
Old Ballantraun 10
Old Ballantruan
Old Ballantruan 10
Old Bardstown
Old Bardstown 101
Old Bardstown Bottled in Bo

Springbank 21 2104
Springbank 21 Rum Cask
Springbank 21 Single Cask
Springbank 21 Single Cask TWE Show Bottling
Springbank 22 Copper Monument Cask 470
Springbank 22 First Fill Sherry Hogshead SMWS
Springbank 22 SMWS 27.110 "Horse harnesses and mescal 'worms'"
Springbank 22 SMWS 27.111
Springbank 24 TWE Show Bottling
Springbank 25
Springbank 35 Dun Bheagan 1970 Cask 1704
Springbank 8 Fresh Sherry Springbank Society
Springbank 8 Society Bottling 2007
Springbank 9 1996 Marsala Wood
Springbank 9 2007 Springbank Society
Springbank 9 Barolo Wood Finish
Springbank 9 Gaja Barolo
Springbank 9 Sauternes Springbank Society
Springbank Alchemist
Springbank Burgundy
Springbank Colonel Stu's Dogpile Reserve
Springbank CV
Springbank private bottling "Colonel Stu's Dogpile Reserve"
Springbank Rundlets & Kilderkins
Springbank Society 14 1995
Springbank Society 8 Fresh Sherry
Springbank Visitor's Miniature 2015
St George Lot 3 US Single Malt
St George Single Malt Batch SM014
St Isidore
St Magdalene 23 19

Tomintoul 40y c1966 Premier Malts
Tomintoul Peaty Tang
Tomintoul That Boutique-y Whisky Company batch 1
TOPO 8 Oak Carolina Whiskey
Tormore 11 1998 Gordon & MacPhail Cask Strength
Tormore 11 2004 Gordon & MacPhail
Tormore 12
Tormore 12 1997 Gordon & MacPhail
Tormore 13 1998 Gordon & Macphail Cask Strength
Tormore 16
Tormore 18 1995 Signatory Vintage
Tormore 19 1990 Signatory
Tormore 19 Whisky Live Tel Aviv 2015 Bottling
Tormore 1988 Wemyss "White Chocolate Torte"
Tormore 20 1992 Berry Bros. & Rudd
Tormore 2002 Cote Rotie Finish
Tormore 2002 Cote Rotie Gordon & MacPhail
Tormore 2004 G&M Cask Strength
Tormore 2004 Gordon & MacPhail Cask Strength
Tormore 22 1992 Signatory Cask Strength
Tormore 25 1984 Cadenhead
Tormore 25 1992 Single Cask First Editions
Tormore 28 1988 Antique Lions of Spirits - Birds Label
Tormore 28 1988 Sansibar
Tormore 28 Master of Malt
Toronto Distillery First Barrels
Toronto Distillery First Barrels Cask Strength
Toronto Distillery Hickory Cask
Toronto Distillery Hi

George Dickel Bottled in Bond
Early Times Bottled in Bond
Heaven Hill 6 Year BiB
EHT Small Batch
EHT Single Barrel
Speyside
Woodford Reserve Malt
Amrut Greedy Angels 10 Cask Strength
Mortlach 2008 10yo (Van Wees - The Ultimate)
Arran 20 Explorer's Series Vol. 1 Brodick Bay
Sagamore Rye Port Finish
George Dickel Single Barrel
Aberlour A'bunadh Batch #63
Longrow 11 Rechar Sherry
Heaven Hill 9 North Star
Ardbeg 17 Elements of Islay - Ar10
Highland Park 16 SMWS 4.250
Laphroaig 10 Cask Strength Batch 10
Ledaig 10 (Valinch & Mallet)
Longrow 17 2001 Single Cask - Chardonnay Finish
1792 225th Anniversary
4Roses 130th Anniversary
Aberlour A'bunadh batch #33
Amrut Sherry Cask Blackadder Raw Cask
Antiquary 21
Ardmore 2008 Signatory
Ardmore Aird Mhor 7 2009 Single Malts of Scotland
Arran 11 Sherry Cask
Arran 13 2014 Oloroso Single Cask Nation
Arran 16 1998 Rest & Be Thankful (cask 98820)
Arran 18 1996 Celebration Of The Cask Càrn Mòr  (cask 2135)
Arran 8 Winter Distillation McGibbon's Provenance 2

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

Some of these have batches specified. In this case batches probably don't have enough variation for us to separate them. In addition, LCBO usually does not list batches so we need to combine them anyway. Let's see if we can find any batch variations:

In [186]:
[print(i)  for i in reviews.whisky.unique() if 'batch' in str.lower(i)]

1792 Full Proof Loch & K(e)y Microbatch
1792 Small Batch
Aberlour A'bunadh batch #06
Aberlour A'bunadh batch #09
Aberlour A'bunadh batch #19
Aberlour A'bunadh batch #26
Aberlour A'bunadh batch #30
Aberlour A'bunadh batch #32
Aberlour A'bunadh batch #34
Aberlour A'Bunadh batch #34
Aberlour A'bunadh batch #35
Aberlour A'bunadh batch #36
Aberlour A'bunadh batch #37
Aberlour A'bunadh batch #38
Aberlour A'bunadh batch #39
Aberlour A'bunadh batch #40
Aberlour A'bunadh batch #41
Aberlour A'bunadh batch #42
Aberlour A'bunadh batch #44
Aberlour A'bunadh batch #45
Aberlour A'bunadh batch #46
Aberlour A'bunadh batch #47
Aberlour A'bunadh batch #49
Aberlour A'bunadh batch #50
Aberlour A'bunadh batch #51
Aberlour A'bunadh Batch #52
Aberlour A'bunadh Batch #53
Aberlour A'bunadh batch #54
Aberlour A'bunadh batch #55
Aberlour A'bunadh batch #56
Aberlour A'bunadh batch #57
Aberlour A'bunadh batch #58
Aberlour A'bunadh batch #59
Aberlour A'bunadh batch #60
Aberlour A'bunadh batch #61
Aberlour A'Bunadh B

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

Here we can see some issues. Lots of whiskies are labelled "Small Batch", which should not be removed. So instead let's make a regex that will remove batch only if it's followed by a number later in the string:

\sbatch.*[0-9].*

And the ones in brackets like (Batch No. 3):

\(batch.*[0-9].*

In [187]:
reviews['whisky'] = reviews['whisky'].str.replace('\sbatch.*[0-9].*', '', case=False, regex=True).str.strip()
reviews['whisky'] = reviews['whisky'].str.replace('\(batch.*[0-9].*', '', case=False, regex=True).str.strip()

In [188]:
[print(i)  for i in reviews.whisky.unique() if 'batch' in str.lower(i)]

1792 Full Proof Loch & K(e)y Microbatch
1792 Small Batch
Alberta Dark Batch
Angel's Envy Bourbon Small Batch
Auchentoshan 14 Cadenhead's Small Batch
Auchentoshan 15 Cadenhead Small Batch
Auchentoshan 17 1999 Cadenhead Small Batch
Auchroisk 24 Cadenhead's Small Batch
Aultmore 1997 Cadenhead's Small Batch
Aultmore-Glenlivet Cadenhead's 17 Small Batch
BenRiach 19 1996 Cadenhead's Small Batch
Benriach 1996 Cadenheads Small Batch
Bernheim Original Small Batch Wheat Whiskey
Bib & Tucker 6 Small Batch
Black Ridge Small Batch
Blair Athol 19 1997 Cadenhead's Small Batch
Blair Athol 19 Cadenhead Small Batch
Booker's Bourbon 2013-6 Roundtable Batch
Booker's Bourbon Big Man Small Batch
Booker's Bourbon Maw Maw's Batch
Booker's Kathleen's Batch
Booker’s Rye Big Time Batch
Bowman Brothers Small Batch
Bowmore 25 Small Batch
Bowmore Small Batch
Bruichladdich 20 1993 Cadenhead's Small Batch
Bruichladdich 24 1992 Cadenhead's Small Batch
Bunnahabhain 39 1976 Cadenhead Small Batch
Bunnahabhain 39 1976 Cad

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [189]:
len(reviews.whisky.unique())

9772

Cadenhead is an independant bottler which LCBO does not carry. So we can filter those out:

In [190]:
reviews = reviews[~reviews['whisky'].str.contains("Cadenhead's", regex=False)]
reviews = reviews[~reviews['whisky'].str.contains("Cadenheads", regex=False)]
reviews = reviews[~reviews['whisky'].str.contains("Cadenhead", regex=False)]

In [191]:
len(reviews.whisky.unique())

9529

In [137]:
[print(i)  for i in reviews.whisky.unique() if 'batch' in str.lower(i)]

1792 Full Proof Loch & K(e)y Microbatch
1792 Small Batch
Alberta Dark Batch
Angel's Envy Bourbon Small Batch
Bernheim Original Small Batch Wheat Whiskey
Bib & Tucker 6 Small Batch
Black Ridge Small Batch
Booker's Bourbon 2013-6 Roundtable Batch
Booker's Bourbon Big Man Small Batch
Booker's Bourbon Maw Maw's Batch
Booker's Kathleen's Batch
Booker’s Rye Big Time Batch
Bowman Brothers Small Batch
Bowmore 25 Small Batch
Bowmore Small Batch
Charles Goodnight Small Batch
Col E.H. Taylor Small Batch
Colonel E.H. Taylor Small Batch
Colonel EH Taylor Small Batch
Corsair Small Batch Triple Smoke American Single Malt
Cyrus Noble Small Batch
Danfield's Private Reserve Small Batch
Devil's River Small Batch Texas Bourbon
Devil’s Share California Small Batch
EHT Small Batch
Elijah Craig Small Batch
Elijah Craig Small Batch (Bayway Liquors Pick)
Elijah Craig Small Batch (HyVee Pick)
Elijah Craig Small Batch - Loch & K(e)y
Elijah Craig Small Batch Moonshine Pick
Elijah Craig Small Batch Oak Liquor Cabi

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

# Join on multiple match file

In [252]:
multiples = pd.read_parquet('data/product_ids.parquet')

Filter out some of the matches that don't make any sense, like ones that aren't whisky:

In [304]:
multiples.head()

,Name,ProductID,RedditWhiskyID,RedditWhiskyName
0,ALBERTA PREMIUM DARK HORSE WHISKY,298083,166,Alberta Premium
1,ALBERTA PREMIUM WHISKY,984,166,Alberta Premium
2,ALBERTA PREMIUM WHISKY,216226,166,Alberta Premium
3,ALBERTA PREMIUM WHISKY (PET),245944,166,Alberta Premium
4,ALBERTA PREMIUM WHISKY (PET),3871,166,Alberta Premium


In [305]:
lcbo.head(2)

,language,itemnumber,itemname,price,productsize,isdiscontinued,islimited,am,ambonusmiles,amexpiration,...,subcategoryname,sellingpackage,sweetnessdescriptor,winestyle,winevarietal,stylebody,stylebody_fr,styleflavour,styleflavour_fr,quantity
1868,en,131870,BUSHMILLS MALT 10 YEAR OLD IRISH WHISKEY,48.95,750,True,True,False,0,NaT,...,Irish Whiskey,bottle,,Subtle & Fruity,Ireland,Subtle,Léger,Fruity,fruité,1
3940,en,133941,1922 HYDE RUM CASK FINISH IRISH WHISKEY,87.85,700,True,True,False,0,NaT,...,,bottle,-,,,,,,,1


In [307]:
lcbo[lcbo['itemnumber'] == 984]

,language,itemnumber,itemname,price,productsize,isdiscontinued,islimited,am,ambonusmiles,amexpiration,...,subcategoryname,sellingpackage,sweetnessdescriptor,winestyle,winevarietal,stylebody,stylebody_fr,styleflavour,styleflavour_fr,quantity
984,en,984,ALBERTA PREMIUM WHISKY,27.2,750,True,True,True,10,2019-07-20,...,Canadian Whisky,bottle,,Medium & Spicy,Canada,Medium,Mi-corsé,Spicy,épicé,1


In [374]:
multiples = pd.read_parquet('data/product_ids.parquet')
multiples['ProductID'] = multiples['ProductID'].astype(int)
multiples = (multiples.set_index('ProductID')
          .join(lcbo.set_index('itemnumber'))
          .reset_index()
          .rename({'index':'itemnumber'},axis='columns')
          .sort_values('RedditWhiskyName')
)

/opt/tljh/user/lib/python3.6/site-packages/pyarrow/pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/opt/tljh/user/lib/python3.6/site-packages/pyarrow/pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
/opt/tljh/user/lib/python3.6/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


Filter out category

In [375]:
print(multiples.shape)
multiples = multiples[multiples['categoryname'] == 'Whisky/Whiskey']
print(multiples.shape)

(952, 48)
(599, 48)


See if we have new single matches

In [376]:
singlematchcategory = multiples.groupby('RedditWhiskyName').count()
singlematchcategory = singlematchcategory[singlematchcategory['itemnumber'] == 1][{}].join(multiples.set_index('RedditWhiskyName'), how='left')

Filter out those single matches to get only multiples again

In [377]:
multiplematch = multiples.groupby('RedditWhiskyName').count()
multiples = (multiplematch[multiplematch['itemnumber'] > 1][{}]
                     .join(multiples.set_index('RedditWhiskyName'), how='left')
                     .reset_index()
                     .rename({'index':'RedditWhiskyName'},axis='columns')
            )

Filter out multiples based on container size

In [378]:
print(multiples.shape)
multiples = multiples[multiples['productsize'] == 750]
print(multiples.shape)

(570, 48)
(369, 48)


In [379]:
singlematchsize = multiples.groupby('RedditWhiskyName').count()
singlematchsize = singlematchsize[singlematchsize['itemnumber'] == 1][{}].join(multiples.set_index('RedditWhiskyName'), how='left')

In [380]:
singlematchsize.shape

(29, 47)

In [381]:
multiplematch = multiples.groupby('RedditWhiskyName').count()
multiples = (multiplematch[multiplematch['itemnumber'] > 1][{}]
                     .join(multiples.set_index('RedditWhiskyName'), how='left')
                     .reset_index()
                     .rename({'index':'RedditWhiskyName'},axis='columns')
            )

In [383]:
multiples

,RedditWhiskyName,itemnumber,Name,RedditWhiskyID,language,itemname,price,productsize,isdiscontinued,islimited,...,subcategoryname,sellingpackage,sweetnessdescriptor,winestyle,winevarietal,stylebody,stylebody_fr,styleflavour,styleflavour_fr,quantity
0,Alberta Premium,984,ALBERTA PREMIUM WHISKY,166,en,ALBERTA PREMIUM WHISKY,27.20,750.0,True,True,...,Canadian Whisky,bottle,,Medium & Spicy,Canada,Medium,Mi-corsé,Spicy,épicé,1.0
1,Alberta Premium,517136,ALBERTA PREMIUM WHISKY (PET),166,en,ALBERTA PREMIUM WHISKY (PET),27.20,750.0,True,True,...,Canadian Whisky,bottle,,,,,,,,1.0
2,Alberta Premium,298083,ALBERTA PREMIUM DARK HORSE WHISKY,166,en,ALBERTA PREMIUM DARK HORSE WHISKY,34.95,750.0,True,True,...,Canadian Whisky,bottle,,Bold & Sweet,Canada,Bold,Puissant,Sweet,doux,1.0
3,Ardbeg & Aultmore Double Barrel,211730,ARDBEG CORRYVRECKAN ISLAY SINGLE MALT SCOTCH W...,276,en,ARDBEG CORRYVRECKAN ISLAY SINGLE MALT SCOTCH W...,200.95,750.0,True,True,...,Scotch Single Malts,bottle,,,Single Malt Scotch,,,,,1.0
4,Ardbeg & Aultmore Double Barrel,539643,ARDBEG AN OA ISLAY SINGLE MALT SCOTCH WHISKY,276,en,ARDBEG AN OA ISLAY SINGLE MALT SCOTCH WHISKY,129.95,750.0,True,True,...,Scotch Single Malts,bottle,,,Whisky - Scotch,,,,,1.0
5,Ardbeg & Aultmore Double Barrel,560474,ARDBEG 10 YEAR OLD ISLAY SCOTCH WHISKY,276,en,ARDBEG 10 YEAR OLD ISLAY SCOTCH WHISKY,100.20,750.0,True,True,...,Scotch Single Malts,bottle,,Bold & Smoky,Scotland Other,Bold,Puissant,Smoky,fumé,1.0
6,Ardbeg & Craigellachie Double Barrel,211730,ARDBEG CORRYVRECKAN ISLAY SINGLE MALT SCOTCH W...,277,en,ARDBEG CORRYVRECKAN ISLAY SINGLE MALT SCOTCH W...,200.95,750.0,True,True,...,Scotch Single Malts,bottle,,,Single Malt Scotch,,,,,1.0
7,Ardbeg & Craigellachie Double Barrel,539643,ARDBEG AN OA ISLAY SINGLE MALT SCOTCH WHISKY,277,en,ARDBEG AN OA ISLAY SINGLE MALT SCOTCH WHISKY,129.95,750.0,True,True,...,Scotch Single Malts,bottle,,,Whisky - Scotch,,,,,1.0
8,Ardbeg & Craigellachie Double Barrel,560474,ARDBEG 10 YEAR OLD ISLAY SCOTCH WHISKY,277,en,ARDBEG 10 YEAR OLD ISLAY SCOTCH WHISKY,100.20,750.0,True,True,...,Scotch Single Malts,bottle,,Bold & Smoky,Scotland Other,Bold,Puissant,Smoky,fumé,1.0
9,Basil Hayden's,638379,BASIL HAYDEN'S TWO BY TWO RYE,921,en,BASIL HAYDEN'S TWO BY TWO RYE,65.25,750.0,True,True,...,,bottle,,,,,,,,1.0


Let's try to figure out which of the multiples we should match:

In [296]:
lcbo[lcbo['itemname'].str.strip()=='ALBERTA PREMIUM WHISKY (PET)']

,language,itemnumber,itemname,price,productsize,isdiscontinued,islimited,am,ambonusmiles,amexpiration,...,subcategoryname,sellingpackage,sweetnessdescriptor,winestyle,winevarietal,stylebody,stylebody_fr,styleflavour,styleflavour_fr,quantity
115929,en,245944,ALBERTA PREMIUM WHISKY (PET),8.55,200,True,True,False,0,NaT,...,Canadian Whisky,bottle,,Medium & Spicy,Canada,Medium,Mi-corsé,Spicy,épicé,1
237096,en,517136,ALBERTA PREMIUM WHISKY (PET),27.20,750,True,True,True,10,2019-07-20,...,Canadian Whisky,bottle,,,,,,,,1
4212,en,54213,ALBERTA PREMIUM WHISKY (PET),61.15,1750,True,True,False,0,NaT,...,Canadian Whisky,bottle,,Medium & Spicy,Canada,Medium,Mi-corsé,Spicy,épicé,1
3871,en,3871,ALBERTA PREMIUM WHISKY (PET),15.25,375,True,True,False,0,NaT,...,Canadian Whisky,bottle,,Medium & Spicy,Canada,Medium,Mi-corsé,Spicy,épicé,1


In [294]:
lcbo[lcbo['itemnumber'] == 517136].itemname

237096    ALBERTA PREMIUM WHISKY (PET)
Name: itemname, dtype: object

In [387]:
multiples

,RedditWhiskyName,itemnumber,Name,RedditWhiskyID,language,itemname,price,productsize,isdiscontinued,islimited,...,subcategoryname,sellingpackage,sweetnessdescriptor,winestyle,winevarietal,stylebody,stylebody_fr,styleflavour,styleflavour_fr,quantity
0,Alberta Premium,984,ALBERTA PREMIUM WHISKY,166,en,ALBERTA PREMIUM WHISKY,27.20,750.0,True,True,...,Canadian Whisky,bottle,,Medium & Spicy,Canada,Medium,Mi-corsé,Spicy,épicé,1.0
1,Alberta Premium,517136,ALBERTA PREMIUM WHISKY (PET),166,en,ALBERTA PREMIUM WHISKY (PET),27.20,750.0,True,True,...,Canadian Whisky,bottle,,,,,,,,1.0
2,Alberta Premium,298083,ALBERTA PREMIUM DARK HORSE WHISKY,166,en,ALBERTA PREMIUM DARK HORSE WHISKY,34.95,750.0,True,True,...,Canadian Whisky,bottle,,Bold & Sweet,Canada,Bold,Puissant,Sweet,doux,1.0
3,Ardbeg & Aultmore Double Barrel,211730,ARDBEG CORRYVRECKAN ISLAY SINGLE MALT SCOTCH W...,276,en,ARDBEG CORRYVRECKAN ISLAY SINGLE MALT SCOTCH W...,200.95,750.0,True,True,...,Scotch Single Malts,bottle,,,Single Malt Scotch,,,,,1.0
4,Ardbeg & Aultmore Double Barrel,539643,ARDBEG AN OA ISLAY SINGLE MALT SCOTCH WHISKY,276,en,ARDBEG AN OA ISLAY SINGLE MALT SCOTCH WHISKY,129.95,750.0,True,True,...,Scotch Single Malts,bottle,,,Whisky - Scotch,,,,,1.0
5,Ardbeg & Aultmore Double Barrel,560474,ARDBEG 10 YEAR OLD ISLAY SCOTCH WHISKY,276,en,ARDBEG 10 YEAR OLD ISLAY SCOTCH WHISKY,100.20,750.0,True,True,...,Scotch Single Malts,bottle,,Bold & Smoky,Scotland Other,Bold,Puissant,Smoky,fumé,1.0
6,Ardbeg & Craigellachie Double Barrel,211730,ARDBEG CORRYVRECKAN ISLAY SINGLE MALT SCOTCH W...,277,en,ARDBEG CORRYVRECKAN ISLAY SINGLE MALT SCOTCH W...,200.95,750.0,True,True,...,Scotch Single Malts,bottle,,,Single Malt Scotch,,,,,1.0
7,Ardbeg & Craigellachie Double Barrel,539643,ARDBEG AN OA ISLAY SINGLE MALT SCOTCH WHISKY,277,en,ARDBEG AN OA ISLAY SINGLE MALT SCOTCH WHISKY,129.95,750.0,True,True,...,Scotch Single Malts,bottle,,,Whisky - Scotch,,,,,1.0
8,Ardbeg & Craigellachie Double Barrel,560474,ARDBEG 10 YEAR OLD ISLAY SCOTCH WHISKY,277,en,ARDBEG 10 YEAR OLD ISLAY SCOTCH WHISKY,100.20,750.0,True,True,...,Scotch Single Malts,bottle,,Bold & Smoky,Scotland Other,Bold,Puissant,Smoky,fumé,1.0
9,Basil Hayden's,638379,BASIL HAYDEN'S TWO BY TWO RYE,921,en,BASIL HAYDEN'S TWO BY TWO RYE,65.25,750.0,True,True,...,,bottle,,,,,,,,1.0


Let's do a string comparison to find how many words are similar to each other.
To do this we can use fuzzywuzzy library, which uses Levenshtein Distance to see how similar two strings are.

In [385]:
from fuzzywuzzy import fuzz

In [400]:
multiples[:2]

,RedditWhiskyName,itemnumber,Name,RedditWhiskyID,language,itemname,price,productsize,isdiscontinued,islimited,...,subcategoryname,sellingpackage,sweetnessdescriptor,winestyle,winevarietal,stylebody,stylebody_fr,styleflavour,styleflavour_fr,quantity
0,Alberta Premium,984,ALBERTA PREMIUM WHISKY,166,en,ALBERTA PREMIUM WHISKY,27.2,750.0,True,True,...,Canadian Whisky,bottle,,Medium & Spicy,Canada,Medium,Mi-corsé,Spicy,épicé,1.0
1,Alberta Premium,517136,ALBERTA PREMIUM WHISKY (PET),166,en,ALBERTA PREMIUM WHISKY (PET),27.2,750.0,True,True,...,Canadian Whisky,bottle,,,,,,,,1.0


In [409]:
fuzzmatch = multiples
fuzzmatch['fuzzratio'] = fuzzmatch.apply(lambda row: fuzz.ratio(row['RedditWhiskyName'],row['Name']), axis='columns')
fuzzmatch['fuzzpartial'] = fuzzmatch.apply(lambda row: fuzz.partial_ratio(row['RedditWhiskyName'],row['Name']), axis='columns')
fuzzmatch['fuzztsort'] = fuzzmatch.apply(lambda row: fuzz.token_sort_ratio(row['RedditWhiskyName'],row['Name']), axis='columns')
fuzzmatch['fuzztset']  = fuzzmatch.apply(lambda row: fuzz.token_set_ratio(row['RedditWhiskyName'],row['Name']), axis='columns')
fuzzmatch[['RedditWhiskyName','Name','fuzzratio','fuzzpartial','fuzztsort','fuzztset']]

,RedditWhiskyName,Name,fuzzratio,fuzzpartial,fuzztsort,fuzztset
0,Alberta Premium,ALBERTA PREMIUM WHISKY,16,20,81,100
1,Alberta Premium,ALBERTA PREMIUM WHISKY (PET),14,20,73,100
2,Alberta Premium,ALBERTA PREMIUM DARK HORSE WHISKY,12,20,62,100
3,Ardbeg & Aultmore Double Barrel,ARDBEG CORRYVRECKAN ISLAY SINGLE MALT SCOTCH W...,15,19,42,42
4,Ardbeg & Aultmore Double Barrel,ARDBEG AN OA ISLAY SINGLE MALT SCOTCH WHISKY,16,19,44,44
5,Ardbeg & Aultmore Double Barrel,ARDBEG 10 YEAR OLD ISLAY SCOTCH WHISKY,20,23,39,39
6,Ardbeg & Craigellachie Double Barrel,ARDBEG CORRYVRECKAN ISLAY SINGLE MALT SCOTCH W...,11,14,47,47
7,Ardbeg & Craigellachie Double Barrel,ARDBEG AN OA ISLAY SINGLE MALT SCOTCH WHISKY,12,14,44,46
8,Ardbeg & Craigellachie Double Barrel,ARDBEG 10 YEAR OLD ISLAY SCOTCH WHISKY,16,17,44,44
9,Basil Hayden's,BASIL HAYDEN'S TWO BY TWO RYE,19,29,65,100


# Check if New Names match
Since we cleaned names, we might have some that are the same as previously:

In [249]:
reviews[['whisky','LCBOName']].drop_duplicates()

,whisky,LCBOName
0,100 Pipers,NaN
1,1792 Bottled In Bond,NaN
2,1792 Bottled In Bond Oak Liquor Cabinet Pick,NaN
3,1792 Full Proof,NaN
20,1792 Full Proof Angel's Beverage,NaN
21,1792 Full Proof Binny's,NaN
22,1792 Full Proof Loch & K(e)y Microbatch,NaN
23,1792 Full Proof Lueken's Store Pick,NaN
24,1792 Full Proof Oak Liquor Cabinet Pick,NaN
25,1792 Full Proof Poison Girl,NaN


# Rejoin on LCBO
Let's see if we have any unjoined whiskies remaining:

In [232]:
reviews.dropna(subset=['ProductID'])[{'whisky','LCBOName'}].drop_duplicates()

,LCBOName,whisky
57,1792 SINGLE BARREL KENTUCKY STRAIGHT BOURBON W...,1792 Single Barrel
63,1792 SMALL BATCH KENTUCKY STRAIGHT BOURBON,1792 Small Batch
91,601 BOURBON,601 Bourbon
107,ABERFELDY 12 YEAR OLD,Aberfeldy 12
135,ABERFELDY 12 YEAR OLD,Aberfeldy 12 Year Old
153,ABERFELDY 21 YEAR OLD HIGHLAND SINGLE MALT SCO...,Aberfeldy 21
164,ABERLOUR 10 YEAR OLD SINGLE MALT SCOTCH WHISKY,Aberlour 10
424,ABERLOUR A'BUNADH SCOTCH WHISKY,Aberlour A'bunadh
717,ALBERTA PREMIUM WHISKY,Alberta Premium 30
719,ALBERTA PREMIUM DARK HORSE WHISKY,Alberta Premium Dark Horse


In [97]:
onematch.head()

,Name,ProductID,RedditWhiskyID,RedditWhiskyName
0,1792 SINGLE BARREL KENTUCKY STRAIGHT BOURBON W...,496729,15,1792 Single Barrel
0,1792 SMALL BATCH KENTUCKY STRAIGHT BOURBON,208918,19,1792 Small Batch
0,601 BOURBON,634519,25,601 Bourbon
0,ABERFELDY 12 YEAR OLD,255281,34,Aberfeldy 12
0,ABERFELDY 12 YEAR OLD,255281,35,Aberfeldy 12 Year Old


In [98]:
reviews.head()

,timestamp,whisky,username,reviewLink,rating,style,pricePaid,date,sub_id,review
reviewID,,,,,,,,,,
0,2012-12-14 10:03:18.000,100 Pipers,merlinblack,http://www.reddit.com/r/Scotch/comments/14uder...,68,Blend,nan,2012-12-14,14uder,My wife and I are on a trip to Thailand to mee...
1,2018-11-04 00:50:38.789,1792 225th Anniversary,scottmotorrad,https://www.reddit.com/r/bourbon/comments/9ssq...,8,Bourbon,nan,2018-10-30,9ssqyk,The 1792 series continues and this time we are...
2,2017-11-17 08:15:21.922,1792 225th Anniversary,WildOscar66,https://www.reddit.com/r/bourbon/comments/7445...,80,Bourbon,nan,2017-10-17,7445vx,Bourbon Review #48 1792 225th Anniversary \n\n...
3,2018-11-04 00:49:14.921,1792 Bottled In Bond,scottmotorrad,https://www.reddit.com/r/bourbon/comments/9ryi...,80,Bourbon,34.99,2018-10-27,9ryi8f,The 1792 review series continues! The Bottled ...
4,2018-12-19 00:38:02.415,1792 Bottled In Bond Oak Liquor Cabinet Pick,scottmotorrad,https://www.reddit.com/r/bourbon/comments/a7jw...,90,Bourbon,nan,2018-12-19,a7jwdd,This week Oak Liquor Cabinet got in the long a...


In [95]:
len(lcbo)

7360